calculating scores from the sampled_paramet.xlsx files output by climraster

only run on sheets that sample a single location, and primarily deal with "occurrence tiers"

In [83]:
import pandas as pd
import xarray as xr
import openpyxl as xl
import os

In [84]:
paramet = 'ros'

file = r"C:\Users\CAMG038492\OneDrive - WSP O365\Documents\Climate Data\John Cabot GIS\Output Samples\sampled_paramet.xlsx".replace("paramet",paramet)
wb = xl.load_workbook(file)


outputfolder = r"C:\Users\CAMG038492\OneDrive - WSP O365\Documents\Climate Data\John Cabot GIS\Output Samples"
outputfile = f"zscores_{paramet}.csv"
loc = os.path.join(outputfolder,outputfile)

wb.sheetnames

['1976-2005', '2006-2035', '2021-2050', '2051-2080', '2071-2100']

In [85]:
sheets = pd.read_excel(file, engine="openpyxl", sheet_name=None)

In [ ]:
def reshape(ssheet):
    sections = {} # dictionary of dictionaries
    for item in ssheet.columns:
        d = {} # dictionary
        if item.startswith("sfc"):
            na,me = item.split("_ann")
            func = me.split("_")[0]
            d[func] = ssheet[item].values[0] # first row
            try:
                sections[na] = sections[na] | d
            except KeyError:
                sections[na] = d
            if 'sfcWindmax-anwindspeed63' in na.split("_"): d['thr'] = 30.0/365 # chosen because winnipeg report
            elif 'sfcWindmax-anwindspeed90' in na.split("_"): d['thr'] = 2.0/365 # chosen because winnipeg report
            elif 'sfcWindmax' in na.split("_"): d['thr'] = 63.0
            else: d['thr'] = 50
        if item.startswith("prfr") or item.startswith("ros"):
            na,me = item.split("_ann")
            func = me.split("_")[0]
            d[func] = ssheet[item].values[1] # second row, using st johns-adjacent since nam44
            try:
                sections[na] = sections[na] | d
            except KeyError:
                sections[na] = d
            if 'prfr' in na.split("_"): d['thr'] = 10.0
            elif 'ros' in na.split("_"): d['thr'] = 1.0
            elif 'ros-anrainonsnow1' in na.split("_"): d['thr'] = 33.0/365 # chosen because historical mean
            elif 'ros-anrainonsnow10' in na.split("_"): d['thr'] = 8.0/365 # chosen because historical mean
            else: d['thr'] = 0
    return sections

d = {}
for years in sheets.keys():
    d = d | reshape(sheets[years])
d

{'ros-anrainonsnow10_CanRCM4_historical_1976-2005': {'max': np.int64(1),
  'thr': 0.021917808219178082,
  'mean': np.float64(0.02410958904109589),
  'min': np.int64(0),
  'std': np.float64(0.150896642257269),
  'sum': np.float64(8.8)},
 'ros-anrainonsnow1_CanRCM4_historical_1976-2005': {'max': np.float64(1.0),
  'thr': 0.09041095890410959,
  'mean': np.float64(0.09086757990867582),
  'min': np.int64(0),
  'std': np.float64(0.2856475503058278),
  'sum': np.float64(33.16666666666666)},
 'ros_CanRCM4_historical_1976-2005': {'max': np.float64(32.26298522949219),
  'thr': 10.0,
  'mean': np.float64(0.7085712552070618),
  'min': np.int64(0),
  'std': np.float64(3.034442663192749),
  'sum': np.float64(258.6285095214844)},
 'ros-anrainonsnow10_CanRCM4_rcp45_2006-2035': {'max': np.int64(1),
  'thr': 0.021917808219178082,
  'mean': np.float64(0.02392694063926941),
  'min': np.int64(0),
  'std': np.float64(0.1482560115210446),
  'sum': np.float64(8.733333333333333)},
 'ros-anrainonsnow1_CanRCM4_r

In [87]:
import scipy.stats
def z2p(z):
    p_value = scipy.stats.norm.sf(abs(z))
    return p_value

def p2l(p):
    if p < .2 and p > 0: l = 1
    elif p < .4: l = 2
    elif p < .6: l = 3
    elif p < .8: l = 4
    elif p < 1: l = 5
    else: raise ValueError(f"invalid p value {p}")
    return l

In [88]:
#thr = 30.0
rowdict = {}
for item in d.keys():
    try:
        mean = d[item]['mean']
    except KeyError:
        #raise ValueError("missing data")
        pass
    try:
        sum = d[item]['sum']
    except KeyError:
        pass
    try:
        std = d[item]['std']
    except KeyError:
        #raise ValueError("missing data")
        pass
    try:
        thr = d[item]['thr']
    except KeyError:
        #raise ValueError("missing data")
        pass
    try:
        zscore = (thr - mean) / std
        pvalue = z2p(zscore)
        invpvalue = z2p(1-zscore)
        score = p2l(pvalue)
        invscore = p2l(invpvalue)
        rowdict[item] = {'mean' : mean, 'std' : std, 'sum' : sum, 'thr' : thr, 'zscore' : zscore,
                         'mean days' : mean*365, 'std days' : std*365,
                         'p value' : pvalue, 'inverted p value' : invpvalue,
                         'climate score' : score, 'inv climate score' : invscore}
    except:
        pass

In [89]:
sdf = pd.DataFrame(rowdict).transpose()
sdf

,mean,std,sum,thr,zscore,mean days,std days,p value,inverted p value,climate score,inv climate score
ros-anrainonsnow10_CanRCM4_historical_1976-2005,0.024110,0.150897,8.800000,0.021918,-0.014525,8.800000,55.077274,4.942055e-01,1.551661e-01,3.0,1.0
ros-anrainonsnow1_CanRCM4_historical_1976-2005,0.090868,0.285648,33.166667,0.090411,-0.001599,33.166667,104.261356,4.993623e-01,1.582688e-01,3.0,1.0
ros_CanRCM4_historical_1976-2005,0.708571,3.034443,258.628510,10.000000,3.061989,258.628508,1107.571572,1.099359e-03,1.960441e-02,1.0,1.0
ros-anrainonsnow10_CanRCM4_rcp45_2006-2035,0.023927,0.148256,8.733333,0.021918,-0.013552,8.733333,54.113444,4.945938e-01,1.553983e-01,3.0,1.0
ros-anrainonsnow1_CanRCM4_rcp45_2006-2035,0.072146,0.255741,26.333333,0.090411,0.071419,26.333333,93.345359,4.715320e-01,1.765532e-01,3.0,1.0
ros_CanRCM4_rcp45_2006-2035,0.651204,3.070195,237.689590,10.000000,3.045017,237.689587,1120.621073,1.163335e-03,2.042658e-02,1.0,1.0
ros-anrainonsnow10_CanRCM4_rcp85_2006-2035,0.020913,0.138974,7.633333,0.021918,0.007228,7.633333,50.725554,4.971163e-01,1.604106e-01,3.0,1.0
ros-anrainonsnow1_CanRCM4_rcp85_2006-2035,0.068950,0.249569,25.166667,0.090411,0.085993,25.166667,91.092665,4.657360e-01,1.803566e-01,3.0,1.0
ros_CanRCM4_rcp85_2006-2035,0.601969,2.913590,219.718704,10.000000,3.225585,219.718643,1063.460246,6.285776e-04,1.302100e-02,1.0,1.0
ros-anrainonsnow10_CanRCM4_rcp45_2021-2050,0.019909,0.134543,7.266667,0.021918,0.014933,7.266667,49.108265,4.940428e-01,1.622956e-01,3.0,1.0


In [90]:
sdf.to_csv(loc)